[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/camenduru's_replace_vae.ipynb)

In [ ]:
!pip install huggingface_hub pytorch_lightning

In [ ]:
# kohya_ss さん、記事ありがとうございます。❤ ❤ https://note.com/kohya_ss/n/nf5893a2e719c 

import torch

def merge_vae(ckpt, vae, output):
  print(f"load checkpoint: {ckpt}")
  model = torch.load(ckpt, map_location="cpu")
  if "state_dict" in model:
    sd = model["state_dict"]
  else:
    sd = model

  full_model = False

  print(f"load VAE: {vae}")
  vae_model = torch.load(vae, map_location="cpu")
  vae_sd = vae_model['state_dict']

  for vae_key in vae_sd:
    if vae_key.startswith("first_stage_model."):
      full_model = True
      break

  for vae_key in vae_sd:
    sd_key = vae_key
    if full_model:
      if not sd_key.startswith("first_stage_model."):
        continue
    else:
      if sd_key not in sd:
        sd_key = "first_stage_model." + sd_key
    if sd_key not in sd:
      print(f"key not exists in model: {vae_key}")
      continue
    sd[sd_key] = vae_sd[vae_key]

  print(f"saving checkpoint to: {output}")
  torch.save(model, output)

!wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt
!wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt
!mkdir /content/swapped
merge_vae("/content/anything-v4.5-pruned.ckpt", "/content/anything-v4.0.vae.pt", "/content/swapped/anything-v4.5-vae-swapped.ckpt")

In [ ]:
import os
from huggingface_hub import create_repo, upload_folder

hf_token=""
repo_id = "ckpt/anything-v4.5-vae-swapped"
path_in_repo = ""
create_repo(repo_id, private=True, token=hf_token)
upload_folder(folder_path="/content/swapped", path_in_repo=path_in_repo, repo_id=repo_id, commit_message=f"anything-v4.5-vae-swapped", token=hf_token)